# 🏛️ Team Project 2: AI-Powered Telegram Trading Bot
**Course:** AI in Finance
**Type:** Production-Ready Financial Assistant

## 1. Project Goal & Context
This project shifts focus from theoretical LLM exploration to **practical, production-oriented development**. We have built a fully functional **Stock Prediction System** accessible via a user-friendly Telegram Bot.

The system integrates:
1.  **Real-Time Data Pipeline**: Fetches live market data (OHLCV) via `yfinance`.
2.  **Generative AI Core**: Utilizes **Google Gemini 2.5-Flash** as the reasoning engine to analyze volatility, macro-factors, and technical indicators.
3.  **Visualization Engine**: Generates dynamic Chart PNGs with volatility thresholds (Green/Red/Orange segments).
4.  **Asynchronous Architecture**: Built on `python-telegram-bot` to handle multiple user requests concurrently.

## 2. Key Capabilities
*   **Target Price Prediction**: Algorithmically extracts specific price targets from AI reasoning.
*   **Confidence Scoring**: Internal validation model estimates the confidence of prediction (e.g., 85%).
*   **Analysis Reports**: Delivers structured "Hedge Fund Style" reports including Risk/Reward ratios and macro context.
---

In [8]:
# ==========================================
# 🟢 STEP 1: ENVIRONMENT SETUP & DEPENDENCIES
# ==========================================
# Phase 1: Infrastructure Initialization.
# We explicitly install 'python-telegram-bot==20.7' to ensure compatibility with our async codebase.
# 'nest_asyncio' is CRITICAL for patching the Event Loop in Google Colab environments.
# 'yfinance' and 'google-genai' act as our Data and Intelligence layers respectively.
!pip install -q python-telegram-bot==20.7 nest_asyncio yfinance google-genai matplotlib pandas

In [9]:
# =========================================================================
# 🟢 STEP 2: TECHNICAL DOCUMENTATION & LIBRARY IMPORTS
# =========================================================================
#
# ARCHITECTURE OVERVIEW:
# This project follows a 4-Layer Architecture to ensure production stability:
#
# 1. CORE INFRASTRUCTURE (os, io, re, logging, asyncio, nest_asyncio):
#    - Handles the underlying OS interactions and asynchronous event loops.
#    - 'nest_asyncio' is verified to be essential for running Telegram's long-polling
#       inside the Google Colab environment without runtime conflicts.
#
# 2. FINANCIAL DATA PIPELINE (pandas, yfinance):
#    - 'yfinance' acts as our primary Oracle for real-time market data retrieval.
#    - 'pandas' manages the time-series dataframes for processing historical candles.
#
# 3. VISUALIZATION ENGINE (matplotlib):
#    - Responsible for generating the diagnostic price curves.
#    - Uses custom logic to segment periods into Growth, Decline, or Stability.
#
# 4. ARTIFICIAL INTELLIGENCE & COMMUNICATION (google-genai, telegram):
#    - 'google-genai' provides the gateway to the Gemini-2.5-Flash model.
#    - 'python-telegram-bot' manages the user-facing interface and command routing.
#
# =========================================================================
# System & Loop Patches
import os
import io
import re
import logging
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
# Data & Charts
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# Generative AI (The Intelligence)
from google import genai
# Telegram Bot API (The Interface)
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, BotCommand
from telegram.ext import Application, CommandHandler, ContextTypes, CallbackQueryHandler
# Apply immediate fixes for Colab environment
nest_asyncio.apply()
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

In [10]:
# =========================================================================
# 🟢 STEP 3: SECURITY & AUTHENTICATION (API KEYS)
# =========================================================================
#
# SECURITY PROTOCOL:
# For production environments, sensitive keys should ideally be stored in
# secrets managers. For this implementation, we use environment variables
# (os.environ) to separate credentials from logic.
#
# 1. TELEGRAM_BOT_TOKEN:
#    - Official credentials from @BotFather. This enables the bot's identity
#      on the Telegram servers.
#
# 2. GEMINI_API_KEY:
#    - Google AI Studio key. Powers the LLM-driven financial analysis.
#
# 3. VALIDATION LOGIC:
#    - The script performs a pre-flight check to ensure both keys are loaded.
#    - If keys are missing, the bot stops immediately to prevent runtime errors.
#
# =========================================================================

import os

# Set global environment variables
os.environ["TELEGRAM_BOT_TOKEN"] = "YOUR_TELEGRAM_BOT_TOKEN_HERE"
os.environ["GEMINI_API_KEY"] = "YOUR_GEMINI_API_KEY_HERE"

# Extract for application use
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Pre-flight validation
if not GEMINI_API_KEY or not TELEGRAM_BOT_TOKEN:
    print("❌ Critical Error: Security credentials (API Keys) are missing!")
else:
    # We display only the first 4 chars for security verification in logs
    print(f"✅ Security Layer Verified. Gemini Linked: {GEMINI_API_KEY[:4]}...")

✅ Security Layer Verified. Gemini Linked: AIza...


In [11]:
# =========================================================================
# 🟢 STEP 4: ARTIFICIAL INTELLIGENCE INITIALIZATION
# =========================================================================
#
# AI INFRASTRUCTURE:
# We utilize the Google Gemini Ecosystem as the analytical core of the bot.
#
# 1. CLIENT INITIALIZATION:
#    - Authenticates the session using the Google GenAI SDK.
#    - Creates a persistent client object to handle inference requests.
#
# 2. MODEL SELECTION (GEMINI-2.5-FLASH):
#    - We use the 'flash' variant because it provides the optimal balance
#      between inference speed and reasoning capabilities for financial data.
#    - It is specifically tuned to handle short-to-medium length contexts
#      (like our 30-day stock histories) with high precision.
#
# 3. ERROR HANDLING (Try-Except):
#    - Production fail-safe: ensures that if the AI API is down or the
#      key is invalid, the bot fails gracefully with a logged message
#      instead of a complete system crash.
#
# =========================================================================

from google import genai

try:
    # Initialize the Google GenAI client
    client = genai.Client(api_key=GEMINI_API_KEY)

    # Standardizing model version for the entire application
    MODEL_NAME = "gemini-2.5-flash"
    print(f"✅ AI Engine Standby. Model: {MODEL_NAME}")

except Exception as e:
    # Critical: This catch-all prevents the bot from starting if AI is unresponsive
    print(f"❌ AI Initialization Error: {e}")

✅ AI Engine Standby. Model: gemini-2.5-flash


In [18]:
# ==========================================
# 🟢 STEP 5: CORE BUSINESS LOGIC - DATA ACQUISITION
# ==========================================

# This text is added to every report to ensure users understand
# that the AI analysis is for information only and not legal financial advice.
DISCLAIMER_TEXT = (
    "\n\n⚠️ Disclaimer: The bot is not responsible for your financial decisions. "
    "This analysis is for educational purposes only. Trading involves risk."
)

# This function fetches historical prices and the current market price from Yahoo Finance.
# It is necessary because the AI needs real factual data to make a realistic prediction.
def get_stock_data(ticker):
    try:
        t = yf.Ticker(ticker)
        # Fetch 3 months of history to give the AI enough technical context.
        hist = t.history(period="3mo", interval="1d")
        if hist.empty:
            return None, 0.0
        try:
            current_price = t.fast_info['last_price']
        except:
          # Fallback if real-time price info is unavailable.
            current_price = hist["Close"].iloc[-1]
        return hist, round(current_price, 2)
    except Exception as e:
        print(f"Fetch Error: {e}")
        return None, 0.0

# This function creates the visual chart showing the past history and the future forecast.
# It is here to give the user a clear, easy-to-read visualization of the predicted trend.
def generate_chart(ticker, history_df, predicted_prices_list, mode):
    try:
        fig, ax = plt.subplots(figsize=(10, 6))
        plt.style.use('bmh')

        # Plotting the real history (Solid blue line)
        hist_len = 30 if mode == "Month" else 7
        subset = history_df.tail(hist_len)
        ax.plot(subset.index, subset["Close"], label=f"History ({hist_len}d)", color="#1f77b4", linewidth=2, marker='o', markersize=3)
        # Calculating dates and points for the predicted curve
        dates = [subset.index[-1]]
        values = [subset["Close"].iloc[-1]]
        current_d = dates[0]

        for val in predicted_prices_list:
            current_d += timedelta(days=1)
            while current_d.weekday() > 4:
                current_d += timedelta(days=1)
            dates.append(current_d)
            values.append(val)

        ax.plot([], [], color='green', linestyle='--', label='Growth (Price goes UP)')
        ax.plot([], [], color='orange', linestyle='--', label='Stable (No significant move <0.1%)')
        ax.plot([], [], color='red', linestyle='--', label='Decline (Price goes DOWN)')

        # Adding descriptive labels to the legend so the user understands the color coding
        for i in range(len(values) - 1):
            start_v, end_v = values[i], values[i+1]
            pct_change = (end_v - start_v) / start_v

            if abs(pct_change) < 0.001:
                seg_color = "orange"
            elif end_v > start_v:
                seg_color = "green"
            else:
                seg_color = "red"

            ax.plot([dates[i], dates[i+1]], [start_v, end_v],
                    color=seg_color, linestyle="--", linewidth=2, marker="o", markersize=4)

        plt.title(f"{ticker}: {mode} Curve", fontsize=14)
        plt.ylabel("Price (USD)")

        plt.legend(loc='upper left', fontsize=9)

        plt.grid(True, alpha=0.3)
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%y'))
        fig.autofmt_xdate(rotation=45)
        plt.tight_layout()
        # Save chart to an in-memory buffer so it can be sent as a photo via Telegram
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=100)
        buf.seek(0)
        plt.close()
        return buf
    except Exception as e:
        print(f"Chart Error: {e}")
        return None

# This function creates instructions for the AI to find relevant financial news events.
# It is here to provide the user with context on what macro-events might affect the bank stocks.
def build_news_prompt():
    today = datetime.now().strftime("%d.%m.%y")
    return f"""
Role: Institutional Macro Researcher.
Task: Identify and analyze major market catalysts for the US Banking Sector (JPM, BAC, WFC) for the 7-day window starting {today}.
Focus: FOMC statements, Treasury yields, Inflation (CPI/PCE), and regulatory updates.
REQUIREMENTS:
1. BANKING IMPACT: For each event, explain specifically how it affects banks (e.g., impact on Net Interest Margin or Loan Demand).
2. VOLATILITY RATING: Assign a status: [CRITICAL], [HIGH], or [MODERATE]. And explain why and how it can affect on people decisions in terms of buying/selling stocks in 2-3 sentances.
3. FORMAT: Strictly use DD.MM.YY.
4. CLOSING: Always end with: "💡 Note: You can see updated news after 3-4 days."
OUTPUT FORMAT:
*it shouldnt be too long so manage the numbers of simbols*
📅 Weekly Watchlist ({today}):
• [DD.MM.YY] - [Event]
  Impact: [High/Med]
  Source: [e.g. Bloomberg]
  Advice: [Tip]
""".strip()

# This function is the "brain" of the bot. It creates the master prompt for the Gemini AI.
# It is here to ensure the AI analyzes technicals, fundamentals, and macro data in a professional way.
def build_prediction_prompt(ticker, price, mode):
    if mode == "Week":
        req = "Provide 7 daily closing prices. PRICES MUST FLUCTUATE DAILY. DO NOT OUTPUT A STRAIGHT LINE."
    elif mode == "Month":
        req = "Provide 30 daily closing prices. SIMULATE REALISTIC MARKET NOISE."
    else:
        req = "Provide 1 predicted closing price."

    return f"""
Role: Senior Financial Analyst (Banking Sector Specialist).
Mission: Provide institutional-grade equity research with a focus on risk management.
Stock: {ticker} (${price}). Task: {mode} Forecast.
ANALYTICAL CONTEXT:
Act as an Institutional Researcher at a top Hedge Fund.
Your forecast must balance quantitative precision (based on price action) with qualitative risks (Macro events).
Do not be overly cautious ("maybe", "could"), be decisive purely based on probability.
ASSUMPTION: The user is a mid-term trader looking for actionable insights, not just passive observation.

DATA SOURCES & CONTEXT (Prioritize):
1. TECHNICALS: Use {price} as the Pivot Point. Consider Support/Resistance levels and psychological barriers (e.g. $100, $50). Mention RSI if the stock feels overbought/oversold.
2. FUNDAMENTALS (Banking Specific): Focus on Net Interest Margin (NIM), Loan Growth outlook, and Credit Default provision trends.
3. MACRO: Federal Reserve stance (Hawkish/Dovish) is CRITICAL for these stocks. Yield Curve shape (10Y-2Y treasury spread).
4. SENTIMENT: Current Fear & Greed Index vibes. Recent headlines from WSJ/Bloomberg regarding the Financial Sector.

CONSTRAINTS & LIMITATIONS:
1. NO HALLUCINATIONS: Do not invent fake "recent news" or events. Use general market knowledge only.
2. VOLATILITY CAP: Do not predict extreme price jumps (>5% daily) unless a major crisis is implied. Banking stocks are typically stable.
3. CONSISTENCY: The "TARGET_PRICE" must logically match the last value in your "PRICES_LIST".
4. NO FINANCIAL ADVICE: Tone must be analytical (educational), not advisory. Never say "You must buy". Say "Buying is recommended".
5. REALISM: If predicting "Hold", the prices in the list should NOT show huge growth. They should be flat/choppy.

REQUIREMENTS:
1. TARGET_PRICE: {req}. Format: [p1, p2...] The single most important price target. price for tommorow/next week(for example if predictions asked on Wed the predicted price shown in the output should forecast for the next Wed)/month (what price will be after next 30days)
2. CONFIDENCE_SCORE: [e.g. 85%] How are you confident in your prediction
3. ANALYSIS: 3-4 clear sentences explaining drivers.
4. PORTFOLIO & INDUSTRY: 2 sentences. Is this stock good for Long-term buy? How does the industry look?
5. NEWS WATCHLIST: What specific event/news should investors watch next week for this stock?
6. STRATEGY: Numbered list (Step 1, Step 2, Step 3). Each step = 1-2 sentences.
7. RECOMMENDATION: Provide a rating (Buy, Sell, or Hold). Include a detailed justification explaining the logic for buying or selling. If the rating is Hold, specify the recommended duration(e.g. hold for 8 days and then try to check the stocks price prediction again...), the strategic reasoning, the expected benefits, and specific market indicators to monitor during the holding period. And if buy (when? now or maybe after some days, if sell when? why?)

OUTPUT FORMAT (Strict, No Markdown Bold **):
TARGET_PRICE: [Insert Number Here]
PRICES_LIST: [p1, p2...]
Confidence score: [Insert %]
Analysis:
Text...
Portfolio & Industry:
Text...
News Watchlist:
Text...
Strategy:
Step 1: ...
Step 2: ...
Recommendation: Rating - detailed justification
""".strip()

In [19]:
# ==========================================
# 🟢 STEP 6: HANDLING COMMANDS & BOT ACTIONS
# ==========================================

# This helper function uses Regular Expressions (Regex) to find number lists in the AI text.
# It is used to convert the AI's "PRICES_LIST" into a Python list that we can plot on the chart.
def extract_prices_robust(text, fallback, count):
    try:
      # Looking for numbers inside square brackets [1, 2, 3...]
        match = re.search(r"\[(.*?)\]", text, re.DOTALL)
        if match:
            nums = re.findall(r"\d+\.\d+|\d+", match.group(1))
            nums = [float(x) for x in nums]
            if len(nums) > 0:
              # Ensure the list matches the required length (7 for week, 30 for month)
                if len(nums) > count: nums = nums[:count]
                while len(nums) < count: nums.append(nums[-1])
                return nums
    except:
        pass
    return [fallback] * count

# This is the main orchestrator for the prediction process.
# It coordinates: 1. Fetching data, 2. AI generation, 3. Text cleaning, 4. Sending the result.
async def run_prediction_flow(message_obj, mode="Tomorrow"):
    await message_obj.reply_text(f"⏳ Analyzing {mode} for JPM, BAC, WFC...")
    stock_list = ["JPM", "BAC", "WFC"]
    for ticker in stock_list:
        try:
            await message_obj.reply_text(f"🔎 Predicting & Analyzing {ticker}...")
            # Step 1: Get market data
            hist, current_price = get_stock_data(ticker)
            if hist is None:
                await message_obj.reply_text(f"❌ No data for {ticker}")
                continue
            # Step 2: Request AI analysis
            prompt = build_prediction_prompt(ticker, current_price, mode)
            resp = client.models.generate_content(model=MODEL_NAME, contents=prompt)
            text = resp.text

            # Step 3: Extract specific "Target Price" using Regex (handles different AI formats)
            target_price = "N/A"
            tp_match = re.search(r"(?:TARGET[_ ]PRICE|PRICE)[:\s]*([\$\d\.]+)", text, re.IGNORECASE)
            if tp_match:
                target_price = tp_match.group(1)
            else:
                fallback_match = re.search(r"[\$]\d+\.\d+", text)
                if fallback_match:
                    target_price = fallback_match.group(0)

            # Step 4: Generate the visual chart
            exp_count = 30 if mode=="Month" else (1 if mode=="Tomorrow" else 7)
            pred_list = extract_prices_robust(text, current_price, exp_count)
            chart = generate_chart(ticker, hist, pred_list, mode)

            # Step 5: Extract "AI Confidence Score"
            confidence = "N/A"
            conf_match = re.search(r"(?:CONFIDENCE[_ ]SCORE|Confidence score)[:\s]*([\d%]+)", text, re.IGNORECASE)
            if conf_match:
              confidence = conf_match.group(1)

            # Step 6: Clean technical tags and brackets from the text for a better user experience
            clean = text.replace("PRICES_LIST:", "").replace("**", "").replace("TARGET_PRICE:", "")
            clean = clean.replace("CONFIDENCE_SCORE:", "")
            clean = re.sub(r"TARGET_PRICE:.*\n?", "", clean, flags=re.IGNORECASE)
            clean = re.sub(r"CONFIDENCE_SCORE:.*\n?", "", clean, flags=re.IGNORECASE)
            clean = re.sub(r"\[.*?\]", "", clean, flags=re.DOTALL)
            clean = re.sub(r"(?:CONFIDENCE[_ ]SCORE|Confidence score)[:\s]*[\d%]+\n?", "", clean, flags=re.IGNORECASE)

            # Deduplicate: Remove the raw price number if it appears at the start of the clean text
            if target_price != "N/A":
                raw_p = target_price.replace("$", "").strip()
                clean = re.sub(r"^\s*[\$]?" + re.escape(raw_p) + r"\s*\n?", "", clean.strip())

            # Step 7: Final message assembly and delivery
            msg = f"📊 {ticker} Report ({mode})\n"
            msg += f"🎯 Predicted Price: {target_price}\n"
            msg += f"🤖 AI Confidence: {confidence}\n\n"
            msg += clean.strip() + DISCLAIMER_TEXT

            await message_obj.reply_text(msg)
            if chart:
                await message_obj.reply_photo(chart)
        except Exception as e:
            await message_obj.reply_text(f"⚠️ Error {ticker}: {e}")

# Command shortcuts for specific timeframes
async def week_cmd(update, context): await run_prediction_flow(update.message, mode="Week")
async def tomorrow_cmd(update, context): await run_prediction_flow(update.message, mode="Tomorrow")
async def month_cmd(update, context): await run_prediction_flow(update.message, mode="Month")

# The landing message for new users when they type /start
async def start(update, context):
    await update.message.reply_text(
        "📈 Stock Prediction Bot\n\n"
        "👋 Welcome!\n\n"
        "Here you can get predictions of stock prices within the banking sector! 🏛️\n\n"
        "Available Commands:\n"
        "🔹/predict - Open Main Menu (Start Here!)\n"
        "🔹/news - Financial Calendar & Events\n"
        "🔹/help - Instructions & Explanation"
    )

# A simple text guide listing all available slash commands
async def help_command(update, context):
    # 🔥 STRICTLY FORMATTED HELP
    await update.message.reply_text(
        "ℹ️ COMMAND REFERENCE:\n\n"
        "/predict - Open Interactive Menu\n"
        "/tomorrow - Next Trading Day Forecast\n"
        "/week - 7-Day Trend Analysis Forecast\n"
        "/month - 30-Day Trend Analysis Forecast\n"
        "/news - Market Calendar & Events\n\n"
        "/jpm - JPMorgan Chase (current price)\n"
        "/bac - Bank of America (current price)\n"
        "/wfc - Wells Fargo (current price)\n\n"
    )
# Command to fetch a list of upcoming economic events that might trigger volatility
async def news_cmd(update, context):
    await update.message.reply_text("📰 Scanning News Sources...")
    try:
        prompt = build_news_prompt()
        resp = client.models.generate_content(model=MODEL_NAME, contents=prompt)
        await update.message.reply_text(resp.text.replace("**", "") + DISCLAIMER_TEXT)
    except Exception as e: await update.message.reply_text(f"Error: {e}")

# Logic for the custom Inline Keyboard (Buttons) for the /predict menu
async def predict_menu(update, context):
    kb = [[InlineKeyboardButton("🔮 Tomorrow", callback_data="pred_tomorrow"), InlineKeyboardButton("📅 Week", callback_data="pred_week"), InlineKeyboardButton("🗓️ Month", callback_data="pred_month")],
          [InlineKeyboardButton("JPM", callback_data="price_JPM"), InlineKeyboardButton("BAC", callback_data="price_BAC"), InlineKeyboardButton("WFC", callback_data="price_WFC")]]
    await update.message.reply_text("🔍 Menu:", reply_markup=InlineKeyboardMarkup(kb))

# Processes clicks on the inline buttons to trigger predictions or price lookups
async def button_handler(update, context):
    q = update.callback_query
    await q.answer()
    if q.data == "pred_tomorrow": await run_prediction_flow(q.message, "Tomorrow")
    elif q.data == "pred_week": await run_prediction_flow(q.message, "Week")
    elif q.data == "pred_month": await run_prediction_flow(q.message, "Month")
    elif q.data.startswith("price_"):
        t = q.data.split("_")[1]
        _, p = get_stock_data(t)
        await q.message.reply_text(f"💵 {t}: ${p}")

# Allows users to check direct prices by typing ticker commands like /JPM
async def ref_price(update, context):
    t = update.message.text.replace("/","").upper()
    _, p = get_stock_data(t)
    await update.message.reply_text(f"💵 {t}: ${p}")

In [20]:
# ==========================================
# 🟢 STEP 7: EXECUTION & BOT DEPLOYMENT
# ==========================================
# This is the entry point of the script. It initializes the Telegram application,
# registers all the command functions we defined earlier, and starts the bot.
if __name__ == "__main__":
    async def main():
        print("🚀 Starting StockBot final version...")
        # Initialize the Telegram Application with your unique Bot Token
        app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()

        # 1. HANDLER REGISTRATION:
        # We tell the bot which Python function to run when a user types a command.
        app.add_handler(CommandHandler("start", start)) # Main welcome message
        app.add_handler(CommandHandler("predict", predict_menu)) # Opens the button menu
        app.add_handler(CommandHandler("week", week_cmd)) # Direct 7-day forecast
        app.add_handler(CommandHandler("month", month_cmd)) # Direct 30-day forecast
        app.add_handler(CommandHandler("tomorrow", tomorrow_cmd)) # Direct next-day forecast
        app.add_handler(CommandHandler("news", news_cmd)) # Economic calendar news
        # Handlers for specific company price checks
        app.add_handler(CommandHandler("jpm", ref_price))
        app.add_handler(CommandHandler("bac", ref_price))
        app.add_handler(CommandHandler("wfc", ref_price))

        app.add_handler(CommandHandler("help", help_command))   # Instructional guide

        # Handler for button clicks in the interactive menu
        app.add_handler(CallbackQueryHandler(button_handler))

        # 2. INITIALIZATION:
        # Prepares the internal state and starts the network connection.
        await app.initialize()
        await app.start()

        # 3. NATIVE MENU REGISTRATION:
        # This part uploads our command list to Telegram's servers.
        # It makes the suggested commands appear when the user clicks the "Menu" button.
        commands = [
            BotCommand("predict", "Main Analysis Menu"),
            BotCommand("news", "Financial Calendar"),
            BotCommand("tomorrow", "Next Day Forecast"),
            BotCommand("week", "7-Day Forecast"),
            BotCommand("month", "30-Day Forecast"),
            BotCommand("jpm", "Price: JPMorgan"),
            BotCommand("bac", "Price: Bank of America"),
            BotCommand("wfc", "Price: Wells Fargo"),
            BotCommand("help", "List all commands")
        ]
        try:
            await app.bot.set_my_commands(commands)
            print("✅ Predictions Menu Updated!")
        except Exception as e:
          # Not critical - the bot still works even if the menu fails to update
            print(f"⚠️ Menu Update Failed (Not Critical): {e}")

        # 4. POLLING:
        # Starts the loop where the bot continuously asks Telegram for new messages.
        await app.updater.start_polling(allowed_updates=Update.ALL_TYPES)
        print("✅ Bot is listening.")
        # Keeps the script running indefinitely
        await asyncio.Event().wait()
    # Handles the difference between running in standard Python vs. Google Colab environment
    try: asyncio.run(main())
    except RuntimeError: loop = asyncio.get_event_loop(); loop.run_until_complete(main()) # Fallback for Colab/Jupyter where the loop is already running

🚀 Starting StockBot final version...
✅ Predictions Menu Updated!
✅ Bot is listening.


Выходные данные были обрезаны до нескольких последних строк (5000).
ERROR:telegram.ext.Updater:Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
ERROR:telegram.ext.Updater:Exception happened while polling for updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_updater.py", line 688, in _network_loop_retry
    if not await action_cb():
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_updater.py", line 384, in polling_action_cb
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_updater.py", line 373, in polling_action_cb
    updates = await self.bot.get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_extbot.py", line 558, in get_updates
    updates = await super().get_updates(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr

KeyboardInterrupt: 